In [2]:
import os
from PIL import Image
import re
from torch.utils.data import Dataset
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
from torchvision.models import *
import numpy as np
import time
from PIL import Image
import torch.utils.data as Data
import torchvision
import torchvision.transforms as transforms

NameError: name '_C' is not defined

In [143]:
imgDir = '/export/hdd/scratch/dataset/imagenet/train/n01694178'
test_imgDir = './cifar_10_images/test_cifar10'
batch_size = 128
batch_size_test  = 100
train_nSamples = 50000
test_nSamples = 10000
init_width = 32
init_height = 32

In [144]:
def is_image_file(filename):
    return any(filename.endswith(extension) for extension in ['.png', '.jpg', '.jpeg', '.PNG', '.JPG', '.JPEG'])

In [145]:
def load_image_path(imgDir):

    all_training_files=os.walk(imgDir)
    train_files=[]
    train_imageNames=[]
    train_nSamples=0
    for path,direction,filelist in all_training_files:
        files = [file for file in filelist if os.path.isfile(os.path.join(path, file))]
        imageNames = [file.split('.')[0] for file in files if is_image_file(file)]
        files = [os.path.join(path, file) for file in files if is_image_file(file)]
        train_files.append(files)
        train_imageNames.append(imageNames)
        train_nSamples=train_nSamples+len(files)
    train_files=sum(train_files,[])
    train_imageNames=sum(train_imageNames,[])
    #print(train_imageNames)
    train_imageNames.sort(key = lambda i:int(re.match(r'(\d+)',i).group()))
    #train_imageNames.sort(key = lambda x: int(x[:-4]))
    train_image_path = []
    for i in range (len(train_imageNames)):
        string = imgDir + '/' + train_imageNames[i] + '.jpg'
        train_image_path.append(string)
    return train_image_path

def load_image_path_test(imgDir):

    all_training_files=os.walk(imgDir)
    train_files=[]
    train_imageNames=[]
    train_nSamples=0
    for path,direction,filelist in all_training_files:
        files = [file for file in filelist if os.path.isfile(os.path.join(path, file))]
        imageNames = [file.split('.')[0] for file in files if is_image_file(file)]
        files = [os.path.join(path, file) for file in files if is_image_file(file)]
        train_files.append(files)
        train_imageNames.append(imageNames)
        train_nSamples=train_nSamples+len(files)
    train_files=sum(train_files,[])
    train_imageNames=sum(train_imageNames,[])
    #print(train_imageNames)
    train_imageNames.sort(key = lambda i:int(re.match(r'(\d+)',i).group()))
    #train_imageNames.sort(key = lambda x: int(x[:-4]))
    train_image_path = []
    for i in range (len(train_imageNames)):
        string = imgDir + '/' + train_imageNames[i] + '.jpg'
        train_image_path.append(string)
    return train_image_path

In [146]:
train_image_path = load_image_path(imgDir)
test_image_path = load_image_path_test(test_imgDir)
train_label = np.load("cifar_10_labels.npy")
test_label = np.load("cifar_10_labels_test.npy")

In [147]:
class listDataset_RAM(Dataset):
    def __init__(self,data, target, nsamples,shape=None, shuffle=True, transform=None, target_transform=None, train=False, seen=0, batch_size=32, num_workers=0):
      
      self.data=data
      self.target=target
      self.nSamples=nsamples
      self.transform = transform
      self.target_transform = target_transform
      self.train = train
      self.shape = shape
      self.seen = seen
      self.batch_size = batch_size
      self.num_workers = num_workers
       
    def __len__(self):
        return self.nSamples

    def __getitem__(self, index):
        #imgpath = self.image_root[index]
        #img = Image.open(imgpath).convert('RGB')
     #print(img)
        img = self.data[index]
        if self.shape is not None:
            img = img.resize(self.shape)
        if self.transform is not None:
            img = self.transform(img)
        label=self.target[index]
        #print(label.type)
        label = torch.from_numpy(np.array(label, dtype = np.int64))
     
        return (img, label)

In [148]:
lr = 0.01
device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

In [149]:
print('==> Preparing data..')
transform_train = transforms.Compose([
    #transforms.ToPILImage(),
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

==> Preparing data..


In [150]:
#train_data = np.zeros([len(train_image_path), 3 ,32,32])
#test_data = np.zeros([len(test_image_path), 3 ,32,32])                       

In [151]:
#print(test_data.shape)
print(train_image_path[4])

./cifar_10_images/train_cifar10/4.jpg


In [152]:
# for i in range (len(train_data)):
#     img = cv2.imread(train_image_path[i])
#     #print(img.shape)
#     #img = np.tranpose(
#     img = np.transpose(img,(2,0,1))
#     #print(img.shape)
#     train_data[i] = np.array(img, dtype = np.float32)

In [153]:
# for i in range (len(test_data)):
#     img = cv2.imread(test_image_path[i])
#     #print(img.shape)
#     #img = np.tranpose(
#     img = np.transpose(img,(2,0,1))
#     #print(img.shape)
#     test_data[i] = np.array(img, dtype = np.float32)

In [154]:
train_data = []
test_data = []
for i in range (len(train_image_path)):
    img = Image.open(train_image_path[i]).convert('RGB')
    train_data.append(img)
    
for i in range (len(test_image_path)):
    img = Image.open(test_image_path[i]).convert('RGB')
    test_data.append(img)

In [155]:
#train_label_tensor=torch.from_numpy(train_label.astype(np.int64))
#test_label_tensor=torch.from_numpy(test_label.astype(np.int64))
#image_tensor=torch.from_numpy(train_data.astype(np.float32))
#image_test_tensor = torch.from_numpy(test_data.astype(np.float32))
#train_data=Data.TensorDataset(image_tensor,train_label_tensor)
#test_data=Data.TensorDataset(image_test_tensor,test_label_tensor)

In [156]:
train_loader = torch.utils.data.DataLoader(
        listDataset_RAM(train_data, train_label, train_nSamples, shape=(init_width, init_height),
                       shuffle=False,
                       transform=transform_train, 
                       train=True, 
                       seen=0,
                       batch_size=batch_size,
                       num_workers=0),
        batch_size=batch_size, shuffle=True, num_workers=8)

In [157]:
test_loader = torch.utils.data.DataLoader(
        listDataset_RAM(test_data, test_label, test_nSamples, shape=(init_width, init_height),
                       shuffle=False,
                       transform=transform_test, 
                       train=False, 
                       seen=0,
                       batch_size=batch_size,
                       num_workers=0),
        batch_size=batch_size_test, shuffle=False, num_workers=8)

In [158]:
net = ResNet18()
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True
    
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr,
                      momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

In [159]:
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    iter = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        iter = iter + 1
        #progress_bar(batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                     #% (train_loss/(batch_idx+1), 100.*correct/total, correct, total))
    print('Loss: %.8f | Acc: %.8f%% (%d/%d)'% (train_loss/(iter * 128), 100.*correct/total, correct, total))
    
def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    iter = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            iter = iter + 1
            #progress_bar(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                         #% (test_loss/(batch_idx+1), 100.*correct/total, correct, total))
    print('Loss: %.8f | Acc: %.8f%% (%d/%d)'% (test_loss/(iter * 100), 100.*correct/total, correct, total))
    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/ckpt_recon1.pth')
        best_acc = acc
        
for epoch in range(start_epoch, start_epoch+200):
    #lr = adjust_learning_rate(epoch)
    start_time = time.time()
    train(epoch)
    test(epoch)
    end_time = time.time()
    print("the time for one epoch is:", end_time - start_time)
    scheduler.step()


Epoch: 0
Loss: 0.01157184 | Acc: 46.02200000% (23011/50000)
Loss: 0.01110253 | Acc: 60.76000000% (6076/10000)
Saving..
the time for one epoch is: 30.619449138641357

Epoch: 1
Loss: 0.00771822 | Acc: 64.97800000% (32489/50000)
Loss: 0.01085528 | Acc: 64.02000000% (6402/10000)
Saving..
the time for one epoch is: 30.908252716064453

Epoch: 2
Loss: 0.00596821 | Acc: 73.35200000% (36676/50000)
Loss: 0.00795556 | Acc: 72.99000000% (7299/10000)
Saving..
the time for one epoch is: 30.215903997421265

Epoch: 3
Loss: 0.00503037 | Acc: 77.74000000% (38870/50000)
Loss: 0.00601846 | Acc: 79.87000000% (7987/10000)
Saving..
the time for one epoch is: 26.67912983894348

Epoch: 4


KeyboardInterrupt: 